In [66]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


13-9. 프로젝트 - Movielens 영화 추천 실습

mkdir -p ~/aiffel/recommendata_iu/data/ml-1m

ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

1) 데이터 준비와 전처리

Dataset

In [67]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [68]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [69]:
ratings = ratings[['user_id', 'movie_id', 'count']]

In [70]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [75]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


2) 분석해 봅시다.

ratings에 있는 유니크한 영화 개수

ratings에 있는 유니크한 사용자 수

가장 인기 있는 영화 30개(인기순)

EDA

In [76]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [77]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [78]:
# 가장 인기있는 영화 30개(인기순)
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

Add ratings

In [79]:
my_favorite = ['Godfather, The (1972)' , 'Back to the Future (1985)' ,'Matrix, The (1999)' ,'Men in Black (1997)' ,'Jurassic Park (1993)']
favorite_movie_id = movies[movies['title'].isin(my_favorite)]
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'count':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie)
ratings.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
476,6041,480,5
847,6041,858,5
1250,6041,1270,5
1539,6041,1580,5
2502,6041,2571,5


In [80]:
favorite_movie_id

,movie_id,title,genre
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
847,858,"Godfather, The (1972)",Action|Crime|Drama
1250,1270,Back to the Future (1985),Comedy|Sci-Fi
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [81]:
movie_data = pd.merge(ratings, movies, on='movie_id')
movie_data

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,Slaughterhouse (1987),Horror
836480,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


4) CSR matrix를 직접 만들어 봅시다.

CSR matrix

In [82]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
476,6041,480,5
847,6041,858,5
1250,6041,1270,5
1539,6041,1580,5


In [83]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

In [84]:
num_user

6040

In [85]:
num_movie

3628

In [86]:
ratings.user_id

0          1
1          1
2          1
3          1
4          1
        ... 
476     6041
847     6041
1250    6041
1539    6041
2502    6041
Name: user_id, Length: 836483, dtype: int64

In [87]:
ratings.movie_id

0       1193
1        661
2        914
3       3408
4       2355
        ... 
476      480
847      858
1250    1270
1539    1580
2502    2571
Name: movie_id, Length: 836483, dtype: int64

In [88]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

Train

In [89]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

In [90]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [91]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [92]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [93]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

좋아하는 영화 선택


Godfather/ Back to the Future/ Matrix/ Men in Black/ Jurassic Park

In [94]:
my_vector, godfather_vector = als_model.user_factors[6041], als_model.item_factors[858]

In [95]:
my_vector

array([-5.21488428e-01,  3.65698367e-01,  5.38365066e-01, -9.18594360e-01,
        1.06364675e-01,  5.82329273e-01,  1.08344591e+00,  1.65835142e-01,
       -4.93130982e-02, -2.42953841e-02, -3.75938147e-01,  1.59314364e-01,
       -9.94466424e-01, -1.66411102e-01, -6.14661038e-01, -1.30239412e-01,
        5.89147955e-02, -3.68898630e-01, -1.74386222e-02, -9.63006437e-01,
        6.93398714e-02, -2.53716469e-01,  1.57950133e-01, -1.08350623e+00,
        1.00864136e+00,  1.14737523e+00,  3.09365600e-01,  6.34021640e-01,
        1.07791126e-02,  4.10859257e-01,  4.36954379e-01, -5.41845560e-01,
       -2.76876420e-01,  2.92541027e-01,  7.10000396e-01, -2.61515051e-01,
        3.15152400e-04, -4.17965829e-01,  3.28058675e-02, -4.87566531e-01,
        8.88535440e-01, -2.08611846e-01, -4.59147185e-01,  1.97949544e-01,
       -1.94758624e-01,  6.31421506e-01, -4.66625184e-01, -4.19151217e-01,
       -8.71323571e-02, -3.25043499e-01,  6.08872175e-01, -4.45453376e-02,
        1.70525104e-01, -

In [96]:
godfather_vector

array([-0.01799835,  0.00632637,  0.00073987, -0.03403559,  0.01895938,
        0.01161432,  0.04272223,  0.03626214, -0.0036663 ,  0.00161238,
        0.00038744,  0.01740954, -0.04377447,  0.00971066, -0.00476695,
        0.00844287,  0.00604917,  0.02725244,  0.0101966 , -0.04450892,
        0.01882148, -0.01433048,  0.00985467, -0.0178862 ,  0.01718012,
        0.01507964,  0.0436704 ,  0.01040962,  0.02491619, -0.02577852,
       -0.02102789, -0.00182972,  0.03516179,  0.01086807,  0.02900275,
        0.003795  , -0.01494853, -0.01689729, -0.0539009 , -0.02982367,
        0.04941583,  0.04494814,  0.02966288,  0.02607063,  0.02610418,
        0.02982944, -0.00210792,  0.0175451 ,  0.01951133,  0.04066649,
        0.04026088,  0.02126892,  0.01002723, -0.00854947, -0.00203177,
       -0.02528859, -0.00035055,  0.03362061,  0.02078374, -0.01386424,
        0.01922092, -0.01485065, -0.00520277, -0.02824732,  0.02267048,
       -0.00133836,  0.05247991, -0.01325364, -0.0207237 , -0.00

In [97]:
np.dot(my_vector, godfather_vector)

0.5907673

In [98]:
toystory_vector = als_model.item_factors[1]
np.dot(my_vector, toystory_vector)

0.07978419

Similar Movie

In [99]:
favorite_movie = 'Godfather, The (1972)'
movie_id = movies[movies['title']=='Godfather, The (1972)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(858, 1.0000001),
 (1221, 0.96279025),
 (2023, 0.57842565),
 (1953, 0.4218285),
 (923, 0.38991803),
 (912, 0.37163168),
 (872, 0.36458078),
 (1787, 0.35554045),
 (2887, 0.34944427),
 (1213, 0.34684724),
 (624, 0.3443785),
 (2695, 0.34300014),
 (3595, 0.3117994),
 (1207, 0.3107928),
 (111, 0.29143322)]

In [100]:
movies[movies['movie_id'].isin([s[0] for s in similar_movie])]

,movie_id,title,genre
109,111,Taxi Driver (1976),Drama|Thriller
619,624,Condition Red (1995),Action|Drama|Thriller
847,858,"Godfather, The (1972)",Action|Crime|Drama
861,872,Aiqing wansui (1994),Drama
900,912,Casablanca (1942),Drama|Romance|War
911,923,Citizen Kane (1941),Drama
1189,1207,To Kill a Mockingbird (1962),Drama
1195,1213,GoodFellas (1990),Crime|Drama
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
1728,1787,Paralyzing Fear: The Story of Polio in America...,Documentary


7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

Favorite Movie

In [101]:
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(589, 0.64535594),
 (1221, 0.52470595),
 (2916, 0.4215576),
 (260, 0.36439812),
 (1210, 0.36282262),
 (110, 0.33766633),
 (1196, 0.336964),
 (1573, 0.3360237),
 (1527, 0.2878201),
 (1240, 0.28710407),
 (648, 0.27963126),
 (2023, 0.27350232),
 (780, 0.26402017),
 (457, 0.24937974),
 (356, 0.24701737),
 (2028, 0.24657926),
 (1544, 0.24584253),
 (2791, 0.22807452),
 (3793, 0.21854919),
 (2716, 0.21133597)]

In [102]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
352,356,Forrest Gump (1994),Comedy|Romance|War
453,457,"Fugitive, The (1993)",Action|Thriller
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
642,648,Mission: Impossible (1996),Action|Adventure|Mystery
770,780,Independence Day (ID4) (1996),Action|Sci-Fi|War
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama


회고

Godfather에 대한 선호도를 측정 0.5907673 

Toystory에 대한 선호도를 측정 0.07978419, 좋아하는 영화 목록에 없었습니다.

Godfather와 유사한 영화를 추천받아

Action|Crime|Drama|Thrillera 장르의 영화를 추천받았습니다.

많은 분들의 도움과 퍼실님의 도움을 받았습니다.

ㅡ감사합니다.

reference 

https://lms.aiffel.io/course/357/node/498/step/3377

https://github.com/JaeHeee/AIFFEL_Project/blob/master/EXPLORATION/EXPLORATION%207.%20%EC%95%84%EC%9D%B4%EC%9C%A0%ED%8C%AC%EC%9D%B4%20%EC%A2%8B%EC%95%84%ED%95%A0%20%EB%A7%8C%ED%95%9C%20%EB%8B%A4%EB%A5%B8%20%EC%95%84%ED%8B%B0%EC%8A%A4%ED%8A%B8%20%EC%B0%BE%EA%B8%B0.ipynb